In [170]:
import os
import re
import cv2
import aocd
import string
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
from aocd import submit
from shapely import Polygon

#Part 1
data = aocd.get_data(day=19, year=2023)
data

'bvh{a>2239:lr,m>657:mqd,x>928:lz,ndn}\nlvc{m<3522:R,m>3836:R,A}\ntg{x<1026:R,R}\ncdj{m<3527:R,x<3765:A,R}\nfgz{a>3380:A,a>2932:A,A}\nnhq{m>3784:R,R}\nsg{x<3555:A,x<3581:R,R}\ntjk{x>2771:ktf,hm}\nrfh{x>603:R,R}\nbdz{x>3391:A,R}\nzx{m>1145:A,a>1075:A,m>529:vrz,tfb}\nkf{a<3575:R,s>3595:R,A}\nczz{x<172:A,a>3342:R,x<192:R,R}\nsxc{a>1352:R,R}\nkvt{a>989:zx,xz}\nvgb{x>3255:bzd,a<3822:gsl,sh}\nvt{m>1457:A,R}\nnhj{a<3658:cz,qmh}\nhjr{a>3504:R,x>2702:rts,s>1565:A,A}\nzn{m<1304:qh,x>3016:hdl,vkz}\nlkq{a<1430:jtt,jch}\nglj{x<3090:cqv,x<3301:A,R}\nvvz{m<2273:A,A}\ndp{x<3685:R,R}\ncx{s>3102:ld,a<282:R,m<872:lrh,jl}\njc{m<2153:R,R}\nbn{a<873:R,R}\nss{a<2112:mqb,s<3068:qr,a<2316:A,bg}\nxpn{x>1071:A,x>943:kdg,nt}\ncsn{m>3093:A,m>2870:jgp,x>3644:vbn,gdt}\nqsq{a>1809:R,a>942:A,m<2294:rmp,A}\nckq{a>3052:xpl,R}\npz{s<2834:R,m<2267:A,A}\nqhb{a>2295:zqv,x<535:rcn,x>839:cxq,xdg}\nsl{s<1641:dl,m>3164:dd,jk}\nlbn{s<2765:R,m<3450:xd,x<3419:fgt,R}\nbdf{m>2402:A,jxs}\nktf{s>572:R,R}\nmb{a<848:fn,x<3270:lrd,trh}\n

In [144]:
data = """px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}"""

In [171]:
raw_rules, raw_parts = data.split('\n\n')
raw_rules = raw_rules.split('\n')
raw_parts = raw_parts.split('\n')
raw_rules, raw_parts
parts = []
for p in raw_parts:
    exec('p = ' + p.replace('{','dict(').replace('}',')'))
    parts += [p]
parts

[{'x': 33, 'm': 357, 'a': 898, 's': 123},
 {'x': 1453, 'm': 618, 'a': 175, 's': 457},
 {'x': 1972, 'm': 3090, 'a': 3668, 's': 2207},
 {'x': 780, 'm': 1219, 'a': 67, 's': 462},
 {'x': 259, 'm': 606, 'a': 1493, 's': 225},
 {'x': 920, 'm': 190, 'a': 381, 's': 1044},
 {'x': 1082, 'm': 2606, 'a': 67, 's': 753},
 {'x': 1790, 'm': 1604, 'a': 2545, 's': 1297},
 {'x': 770, 'm': 1139, 'a': 896, 's': 812},
 {'x': 250, 'm': 2173, 'a': 1294, 's': 2757},
 {'x': 107, 'm': 1176, 'a': 2753, 's': 382},
 {'x': 409, 'm': 2, 'a': 259, 's': 2158},
 {'x': 1207, 'm': 282, 'a': 1739, 's': 1328},
 {'x': 283, 'm': 807, 'a': 1577, 's': 740},
 {'x': 420, 'm': 978, 'a': 1691, 's': 3093},
 {'x': 616, 'm': 1493, 'a': 2390, 's': 318},
 {'x': 245, 'm': 111, 'a': 841, 's': 21},
 {'x': 109, 'm': 584, 'a': 3124, 's': 296},
 {'x': 3775, 'm': 334, 'a': 862, 's': 461},
 {'x': 870, 'm': 1050, 'a': 293, 's': 5},
 {'x': 194, 'm': 24, 'a': 564, 's': 66},
 {'x': 431, 'm': 210, 'a': 1053, 's': 664},
 {'x': 40, 'm': 320, 'a': 497, 

In [172]:
rules = {}
for r in raw_rules:
    key, wf = r[:-1].split('{')
    wf = wf.split(',')
    rules[key] = wf
rules

{'bvh': ['a>2239:lr', 'm>657:mqd', 'x>928:lz', 'ndn'],
 'lvc': ['m<3522:R', 'm>3836:R', 'A'],
 'tg': ['x<1026:R', 'R'],
 'cdj': ['m<3527:R', 'x<3765:A', 'R'],
 'fgz': ['a>3380:A', 'a>2932:A', 'A'],
 'nhq': ['m>3784:R', 'R'],
 'sg': ['x<3555:A', 'x<3581:R', 'R'],
 'tjk': ['x>2771:ktf', 'hm'],
 'rfh': ['x>603:R', 'R'],
 'bdz': ['x>3391:A', 'R'],
 'zx': ['m>1145:A', 'a>1075:A', 'm>529:vrz', 'tfb'],
 'kf': ['a<3575:R', 's>3595:R', 'A'],
 'czz': ['x<172:A', 'a>3342:R', 'x<192:R', 'R'],
 'sxc': ['a>1352:R', 'R'],
 'kvt': ['a>989:zx', 'xz'],
 'vgb': ['x>3255:bzd', 'a<3822:gsl', 'sh'],
 'vt': ['m>1457:A', 'R'],
 'nhj': ['a<3658:cz', 'qmh'],
 'hjr': ['a>3504:R', 'x>2702:rts', 's>1565:A', 'A'],
 'zn': ['m<1304:qh', 'x>3016:hdl', 'vkz'],
 'lkq': ['a<1430:jtt', 'jch'],
 'glj': ['x<3090:cqv', 'x<3301:A', 'R'],
 'vvz': ['m<2273:A', 'A'],
 'dp': ['x<3685:R', 'R'],
 'cx': ['s>3102:ld', 'a<282:R', 'm<872:lrh', 'jl'],
 'jc': ['m<2153:R', 'R'],
 'bn': ['a<873:R', 'R'],
 'ss': ['a<2112:mqb', 's<3068:qr', 

In [147]:
def eval_woof(wf, x,m,a,s):
    if ':' not in wf:
        return wf
    
    ev, next_wf = wf.split(':')
    ldict = {}
    exec(f'bool_ = {ev}',locals(),ldict)
    bool_ = ldict['bool_']
    if bool_:
        return next_wf
    else:
        return None
    
def eval_barks(barks, part):
    for woof in barks:
        res = eval_woof(woof, **part)
        if res == 'R':
            return False
        elif res == 'A':
            return True
        elif res is None:
            continue
        return eval_barks(rules[res],part)
    
_sum = 0
for p in parts:
    accepted = eval_barks(rules['in'],p)
    if accepted:
        _sum += sum([val for k,val in p.items()])
_sum

19114

In [148]:
start.total

256000000000000

In [173]:
class Ranger():
    def __init__(self, xmas):
        self.xmas = xmas
        self.rejected = False
        
    @property
    def total(self):
        return np.product([len(v) for k,v in self.xmas.items()])

    def split_on_woof(self, woof):
        if ':' not in woof:
            return [(woof, self)]
        
        rule, true_woof = woof.split(':')
        
        char, symb, num = rule[0],rule[1],rule[2:]
        num = int(num)
        
        low_ranger = Ranger(self.xmas.copy())
        high_ranger = Ranger(self.xmas.copy())
        
        
        if symb == '>':
            low_ranger.xmas[char] = self.xmas[char][self.xmas[char] <= num]
            high_ranger.xmas[char] = self.xmas[char][self.xmas[char] > num]
            return [(true_woof, high_ranger), (None, low_ranger)]
        if symb == '<':
            low_ranger.xmas[char] = self.xmas[char][self.xmas[char] < num]
            high_ranger.xmas[char] = self.xmas[char][self.xmas[char] >= num]
            return [(None, high_ranger), (true_woof, low_ranger)]
        
    def __repr__(self):
        
        pretty_xmas = {'x':list(self.xmas['x'][[0,-1]]),
                      'm':list(self.xmas['m'][[0,-1]]),
                      'a':list(self.xmas['a'][[0,-1]]),
                      's':list(self.xmas['s'][[0,-1]]),}
        return str(pretty_xmas)


start.split_on_woof('a>10:ab')

[('ab', {'x': [1, 4000], 'm': [1, 4000], 'a': [11, 4000], 's': [1, 4000]}),
 (None, {'x': [1, 4000], 'm': [1, 4000], 'a': [1, 10], 's': [1, 4000]})]

In [174]:
def driver(barks, ranger):
    accepted_rangers = []
    print('Starting Driver...')
    print('  Barks', barks)
    print('  Ranger', ranger)
    
    woof = barks[0]

    print('  Woof',woof)
    true_woofs_and_next_rangers = ranger.split_on_woof(woof)
    print('  Splits',true_woofs_and_next_rangers)
    for true_woof,next_ranger in true_woofs_and_next_rangers:
        if true_woof == 'R':
            print('rejected')
            continue
        elif true_woof == 'A':
            accepted_rangers += [next_ranger]
        elif true_woof is None:
            print(f'  Was none, evaluating {barks[1:]}')
            accepted_rangers += driver(barks[1:], next_ranger)
        else:
            print(f'  Was True, evaluating {true_woof}')
            accepted_rangers += driver(rules[true_woof], next_ranger)

    return accepted_rangers

start = Ranger({'x':np.array(range(1,4001)),
                 'm':np.array(range(1,4001)),
                 'a':np.array(range(1,4001)),
                 's':np.array(range(1,4001))})

accepted_rangers = driver(rules['in'],start)
accepted_rangers

Starting Driver...
  Barks ['x<2351:rx', 'm>2575:dvb', 'fnq']
  Ranger {'x': [1, 4000], 'm': [1, 4000], 'a': [1, 4000], 's': [1, 4000]}
  Woof x<2351:rx
  Splits [(None, {'x': [2351, 4000], 'm': [1, 4000], 'a': [1, 4000], 's': [1, 4000]}), ('rx', {'x': [1, 2350], 'm': [1, 4000], 'a': [1, 4000], 's': [1, 4000]})]
  Was none, evaluating ['m>2575:dvb', 'fnq']
Starting Driver...
  Barks ['m>2575:dvb', 'fnq']
  Ranger {'x': [2351, 4000], 'm': [1, 4000], 'a': [1, 4000], 's': [1, 4000]}
  Woof m>2575:dvb
  Splits [('dvb', {'x': [2351, 4000], 'm': [2576, 4000], 'a': [1, 4000], 's': [1, 4000]}), (None, {'x': [2351, 4000], 'm': [1, 2575], 'a': [1, 4000], 's': [1, 4000]})]
  Was True, evaluating dvb
Starting Driver...
  Barks ['s<2516:dvp', 'pgg']
  Ranger {'x': [2351, 4000], 'm': [2576, 4000], 'a': [1, 4000], 's': [1, 4000]}
  Woof s<2516:dvp
  Splits [(None, {'x': [2351, 4000], 'm': [2576, 4000], 'a': [1, 4000], 's': [2516, 4000]}), ('dvp', {'x': [2351, 4000], 'm': [2576, 4000], 'a': [1, 4000],

  Ranger {'x': [3118, 3187], 'm': [2576, 3324], 'a': [848, 1685], 's': [1, 2515]}
  Woof x>3159:A
  Splits [('A', {'x': [3160, 3187], 'm': [2576, 3324], 'a': [848, 1685], 's': [1, 2515]}), (None, {'x': [3118, 3159], 'm': [2576, 3324], 'a': [848, 1685], 's': [1, 2515]})]
  Was none, evaluating ['A']
Starting Driver...
  Barks ['A']
  Ranger {'x': [3118, 3159], 'm': [2576, 3324], 'a': [848, 1685], 's': [1, 2515]}
  Woof A
  Splits [('A', {'x': [3118, 3159], 'm': [2576, 3324], 'a': [848, 1685], 's': [1, 2515]})]
  Was True, evaluating fn
Starting Driver...
  Barks ['a<402:R', 'm>3238:A', 'A']
  Ranger {'x': [3118, 3415], 'm': [2576, 4000], 'a': [1, 847], 's': [1, 2515]}
  Woof a<402:R
  Splits [(None, {'x': [3118, 3415], 'm': [2576, 4000], 'a': [402, 847], 's': [1, 2515]}), ('R', {'x': [3118, 3415], 'm': [2576, 4000], 'a': [1, 401], 's': [1, 2515]})]
  Was none, evaluating ['m>3238:A', 'A']
Starting Driver...
  Barks ['m>3238:A', 'A']
  Ranger {'x': [3118, 3415], 'm': [2576, 4000], 'a': [

  Barks ['x<3019:R', 'R']
  Ranger {'x': [2351, 3371], 'm': [957, 1216], 'a': [2687, 4000], 's': [2828, 3342]}
  Woof x<3019:R
  Splits [(None, {'x': [3019, 3371], 'm': [957, 1216], 'a': [2687, 4000], 's': [2828, 3342]}), ('R', {'x': [2351, 3018], 'm': [957, 1216], 'a': [2687, 4000], 's': [2828, 3342]})]
  Was none, evaluating ['R']
Starting Driver...
  Barks ['R']
  Ranger {'x': [3019, 3371], 'm': [957, 1216], 'a': [2687, 4000], 's': [2828, 3342]}
  Woof R
  Splits [('R', {'x': [3019, 3371], 'm': [957, 1216], 'a': [2687, 4000], 's': [2828, 3342]})]
rejected
rejected
  Was none, evaluating ['a>3313:R', 'pfj']
Starting Driver...
  Barks ['a>3313:R', 'pfj']
  Ranger {'x': [2351, 3371], 'm': [1, 956], 'a': [2687, 4000], 's': [2828, 3342]}
  Woof a>3313:R
  Splits [('R', {'x': [2351, 3371], 'm': [1, 956], 'a': [3314, 4000], 's': [2828, 3342]}), (None, {'x': [2351, 3371], 'm': [1, 956], 'a': [2687, 3313], 's': [2828, 3342]})]
rejected
  Was none, evaluating ['pfj']
Starting Driver...
  Bark

  Barks ['R']
  Ranger {'x': [3132, 4000], 'm': [1705, 2206], 'a': [270, 454], 's': [2914, 4000]}
  Woof R
  Splits [('R', {'x': [3132, 4000], 'm': [1705, 2206], 'a': [270, 454], 's': [2914, 4000]})]
rejected
  Was True, evaluating lkg
Starting Driver...
  Barks ['x>3557:R', 's>3305:A', 'R']
  Ranger {'x': [3132, 4000], 'm': [1705, 2575], 'a': [1, 269], 's': [2914, 4000]}
  Woof x>3557:R
  Splits [('R', {'x': [3558, 4000], 'm': [1705, 2575], 'a': [1, 269], 's': [2914, 4000]}), (None, {'x': [3132, 3557], 'm': [1705, 2575], 'a': [1, 269], 's': [2914, 4000]})]
rejected
  Was none, evaluating ['s>3305:A', 'R']
Starting Driver...
  Barks ['s>3305:A', 'R']
  Ranger {'x': [3132, 3557], 'm': [1705, 2575], 'a': [1, 269], 's': [2914, 4000]}
  Woof s>3305:A
  Splits [('A', {'x': [3132, 3557], 'm': [1705, 2575], 'a': [1, 269], 's': [3306, 4000]}), (None, {'x': [3132, 3557], 'm': [1705, 2575], 'a': [1, 269], 's': [2914, 3305]})]
  Was none, evaluating ['R']
Starting Driver...
  Barks ['R']
  Ranger

Starting Driver...
  Barks ['m>1349:bsr', 'm<797:A', 'jvb']
  Ranger {'x': [2351, 2628], 'm': [1, 2575], 'a': [388, 768], 's': [2281, 3031]}
  Woof m>1349:bsr
  Splits [('bsr', {'x': [2351, 2628], 'm': [1350, 2575], 'a': [388, 768], 's': [2281, 3031]}), (None, {'x': [2351, 2628], 'm': [1, 1349], 'a': [388, 768], 's': [2281, 3031]})]
  Was True, evaluating bsr
Starting Driver...
  Barks ['a<526:A', 'A']
  Ranger {'x': [2351, 2628], 'm': [1350, 2575], 'a': [388, 768], 's': [2281, 3031]}
  Woof a<526:A
  Splits [(None, {'x': [2351, 2628], 'm': [1350, 2575], 'a': [526, 768], 's': [2281, 3031]}), ('A', {'x': [2351, 2628], 'm': [1350, 2575], 'a': [388, 525], 's': [2281, 3031]})]
  Was none, evaluating ['A']
Starting Driver...
  Barks ['A']
  Ranger {'x': [2351, 2628], 'm': [1350, 2575], 'a': [526, 768], 's': [2281, 3031]}
  Woof A
  Splits [('A', {'x': [2351, 2628], 'm': [1350, 2575], 'a': [526, 768], 's': [2281, 3031]})]
  Was none, evaluating ['m<797:A', 'jvb']
Starting Driver...
  Barks [

  Splits [(None, {'x': [1, 633], 'm': [1055, 2233], 'a': [2569, 4000], 's': [1976, 4000]}), ('hf', {'x': [1, 633], 'm': [1055, 2233], 'a': [2569, 4000], 's': [1, 1975]})]
  Was none, evaluating ['pk']
Starting Driver...
  Barks ['pk']
  Ranger {'x': [1, 633], 'm': [1055, 2233], 'a': [2569, 4000], 's': [1976, 4000]}
  Woof pk
  Splits [('pk', {'x': [1, 633], 'm': [1055, 2233], 'a': [2569, 4000], 's': [1976, 4000]})]
  Was True, evaluating pk
Starting Driver...
  Barks ['a>3350:A', 'm<1581:fcn', 'A']
  Ranger {'x': [1, 633], 'm': [1055, 2233], 'a': [2569, 4000], 's': [1976, 4000]}
  Woof a>3350:A
  Splits [('A', {'x': [1, 633], 'm': [1055, 2233], 'a': [3351, 4000], 's': [1976, 4000]}), (None, {'x': [1, 633], 'm': [1055, 2233], 'a': [2569, 3350], 's': [1976, 4000]})]
  Was none, evaluating ['m<1581:fcn', 'A']
Starting Driver...
  Barks ['m<1581:fcn', 'A']
  Ranger {'x': [1, 633], 'm': [1055, 2233], 'a': [2569, 3350], 's': [1976, 4000]}
  Woof m<1581:fcn
  Splits [(None, {'x': [1, 633], 'm

[{'x': [3735, 4000], 'm': [2576, 4000], 'a': [2421, 4000], 's': [3447, 3801]},
 {'x': [3911, 4000], 'm': [2576, 4000], 'a': [1571, 2420], 's': [3914, 4000]},
 {'x': [3566, 4000], 'm': [3417, 4000], 'a': [2212, 2420], 's': [3447, 3497]},
 {'x': [3566, 4000], 'm': [2576, 4000], 'a': [1571, 2023], 's': [3447, 3580]},
 {'x': [3566, 4000], 'm': [3522, 3836], 'a': [1571, 4000], 's': [2516, 3446]},
 {'x': [3542, 3565], 'm': [2576, 4000], 'a': [1877, 2455], 's': [3101, 4000]},
 {'x': [3542, 3565], 'm': [2576, 4000], 'a': [2093, 2455], 's': [2814, 3100]},
 {'x': [3542, 3565], 'm': [3106, 4000], 'a': [2093, 2455], 's': [2516, 2813]},
 {'x': [3542, 3565], 'm': [2576, 3105], 'a': [2093, 2455], 's': [2516, 2813]},
 {'x': [3533, 3541], 'm': [2576, 4000], 'a': [1931, 2455], 's': [3767, 4000]},
 {'x': [3533, 3541], 'm': [2576, 4000], 'a': [1571, 2066], 's': [3302, 3766]},
 {'x': [3521, 3532], 'm': [2576, 4000], 'a': [1571, 2455], 's': [2516, 4000]},
 {'x': [3461, 3520], 'm': [2576, 3169], 'a': [1571, 

In [155]:
rules

{'px': ['a<2006:qkq', 'm>2090:A', 'rfg'],
 'pv': ['a>1716:R', 'A'],
 'lnx': ['m>1548:A', 'A'],
 'rfg': ['s<537:gd', 'x>2440:R', 'A'],
 'qs': ['s>3448:A', 'lnx'],
 'qkq': ['x<1416:A', 'crn'],
 'crn': ['x>2662:A', 'R'],
 'in': ['s<1351:px', 'qqz'],
 'qqz': ['s>2770:qs', 'm<1801:hdj', 'R'],
 'gd': ['a>3333:R', 'R'],
 'hdj': ['m>838:A', 'pv']}

In [175]:
sum([ranger.total for ranger in accepted_rangers])

130262715574114

In [165]:
167409079868000

167409079868000